# LSTM PyTorch Lightning

In [1]:
import numpy as np
import datetime
now = datetime.datetime.now()

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data as utils
import torch.optim as optim
from torchvision import datasets, transforms

import matplotlib.pyplot as plt
from tqdm.notebook import tqdm

In C:\Users\zankyo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The text.latex.preview rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\zankyo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The mathtext.fallback_to_cm rcparam was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\zankyo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: Support for setting the 'mathtext.fallback_to_cm' rcParam is deprecated since 3.3 and will be removed two minor releases later; use 'mathtext.fallback : 'cm' instead.
In C:\Users\zankyo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_test.mplstyle: 
The validate_bool_maybe_none function was deprecated in Matplotlib 3.3 and will be removed two minor releases later.
In C:\Users\zankyo\Anaconda3\lib\site-packages\matplotlib\mpl-data\stylelib\_classic_tes

In [2]:
import pytorch_lightning as pl

PyTorch Lightningに関するエラーについて<br>
`pip install --ignore-installed --upgrade tensorflow-gpu==1.15.0`
https://github.com/tensorflow/tensorflow/issues/35208

In [3]:
# GPU check

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("使用デバイス:", device)

# Dataset

outname ="lstm"

outdir = 'C:/Users/zankyo/Desktop/yamamoto/model/' +outname+ now.strftime('%Y%m%d_%H%M%S')
print(outdir)

audio_len = 2**15
usedata_num = 15000
sample_rate = 16000

batch_size = 2

datasets_dir = 'C:/Users/zankyo/Desktop/yamamoto/datasets/datasets129x257.npz'
datasets = np.load(datasets_dir)

data, label = datasets['data'], datasets['label']

data_list = []
for i in tqdm(range(len(data))):
    data_list.append(data[i])
    
label_list = []
for i in tqdm(range(len(data))):
    label_list.append(label[i])
    
tensor_data = torch.stack([torch.Tensor(i) for i in data_list])
tensor_label = torch.stack([torch.Tensor(i) for i in label_list])

mydataset = utils.TensorDataset(tensor_data,tensor_label)

train_dataset, val_dataset,test_dataset = utils.random_split(mydataset,[12000,2500,500])

# Dataloader

train_dataloader = utils.DataLoader(train_dataset,batch_size=batch_size,shuffle=True)
val_dataloader = utils.DataLoader(val_dataset,batch_size=batch_size,shuffle=True)
test_dataloader = utils.DataLoader(test_dataset,batch_size=batch_size,shuffle=True)

#辞書型変数にまとめる
dataloaders_dict = {"train":train_dataloader,
                    "val":val_dataloader,
                    "test":test_dataloader}


# 動作の確認
"""
Batch, Freq(Feature), Time(Sequence)
"""

batch_iterator = iter(dataloaders_dict["train"])
datas,labels=next(batch_iterator)
print(datas.size())
print(labels.size())

使用デバイス: cuda
C:/Users/zankyo/Desktop/yamamoto/model/lstm20201116_212038


C:\Users\zankyo\Anaconda3\lib\site-packages\ipykernel_launcher.py:32: UserWarning: Casting complex values to real discards the imaginary part (Triggered internally at  ..\aten\src\ATen\native\Copy.cpp:162.)


torch.Size([2, 129, 257])
torch.Size([2, 129, 257])


In [4]:
feat_size = data.shape[1] #129
sequence_len = data.shape[2] #257
lstm_layer = 1
hidden_layer =129

class Net(pl.LightningModule):
    def __init__(self):
        
        super(Net,self).__init__()
        self.seq_len = sequence_len
        self.feature_size = feat_size
        self.output_size = feat_size
        self.hidden_layer_size = hidden_layer #隠れ層のサイズ
        self.lstm_layers = lstm_layer #LSTMのレイヤー数
        
        self.encoder = nn.Sequential(nn.LSTM(self.feature_size,
                                           self.hidden_layer_size,
                                           num_layers = self.lstm_layers),
                                    nn.ReLU(),
                                   nn.Linear(self.hidden_layer_size,self.output_size),
                                   nn.Sigmoid())
    def forword(self,x):
        embedding = self.encoder(x)
        return embedding        
        
    def configure_optimizers(self):# Optimaizerの設定
        optimizer = torch.optim.Adam(self.parameters(), lr=1e-3)
        return optimizer
    

    def training_step(self, batch, batch_idx):# 完全なトレーニングループ
        # training_stepでは訓練ループを定義していて、これはforwardとは独立しています
        x, y = batch
        z = self.encoder(x)  # forwardを呼び出す self(x) でもよい
        loss = nn.L1Loss(z, y)
#         loss = F.mse_loss(z, y)
        self.log("train_loss", loss)
        return loss
    
    def validation_step(self, val_batch, batch_idx):#完全な検証ループ
        # 検証ステップ
        x, y = val_batch
        z = self.encoder(x)
        loss = nn.L1Loss(z, y)
#         loss = F.mse_loss(z, y)
        self.log("val_loss", loss)
        return loss

In [ ]:
#モデルの呼び出しと初期化
net =Net()# 学習モデルのインスタンス化
#トレーニング開始
trainer = pl.Trainer(max_epochs=5, gpus=1)# 学習用のインスタンス化と学習の設定

trainer.fit(net, train_dataloader,val_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
